# 스마트팜 빅데이터 수집
스마트팜 빅데이터 OPEN API를 통해 데이터를 수집한다.<br>
홈페이지: https://www.smartfarmkorea.net/front/open/list.do?menuId=M01060201#con

In [2]:
import os
import requests
import json
import pandas as pd
from tqdm.notebook import tqdm

tomato_code = '080300'
serviceKey = '923f389b2ca44295a07f14d1828aee0e'

## 전체 농가 정보 조회 → 토마토 농가 추출

![](img/identityDataList.png)

In [2]:
getIdentityDataList = 'http://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getIdentityDataList/'+serviceKey

response = requests.get(getIdentityDataList)
temp = response.text
identityDataList = json.loads(temp)

In [3]:
identityDataList[0]

{'statusCode': '00',
 'statusMessage': 'NORMAL_CODE',
 'userId': 'LSDS_001',
 'facilityId': 'LSDS_001_01',
 'addressName': '경상남도 하동군',
 'itemCode': '413000'}

In [21]:
len(identityDataList)

534

In [3]:
tomatoIdentityList = [identityDataList[i] for i in range(len(identityDataList)) if identityDataList[i]['itemCode']==tomato_code]
tomatoIdentityList

[{'statusCode': None,
  'statusMessage': None,
  'userId': 'PFS_0000001',
  'facilityId': 'PFS_0000001_01',
  'addressName': '경상남도 사천시',
  'itemCode': '080300'},
 {'statusCode': None,
  'statusMessage': None,
  'userId': 'PFS_0000002',
  'facilityId': 'PFS_0000002_01',
  'addressName': '전라북도 김제시',
  'itemCode': '080300'},
 {'statusCode': None,
  'statusMessage': None,
  'userId': 'PFS_0000003',
  'facilityId': 'PFS_0000003_01',
  'addressName': '경상남도 함안군',
  'itemCode': '080300'},
 {'statusCode': None,
  'statusMessage': None,
  'userId': 'PFS_0000004',
  'facilityId': 'PFS_0000004_01',
  'addressName': '경상남도 진주시',
  'itemCode': '080300'},
 {'statusCode': None,
  'statusMessage': None,
  'userId': 'PFS_0000006',
  'facilityId': 'PFS_0000006_01',
  'addressName': '전라북도 순창군',
  'itemCode': '080300'},
 {'statusCode': None,
  'statusMessage': None,
  'userId': 'PFS_0000007',
  'facilityId': 'PFS_0000007_01',
  'addressName': '전라남도 화순군',
  'itemCode': '080300'},
 {'statusCode': None,
  'sta

In [4]:
identity_df = pd.DataFrame(columns=['userId','facilityId','addressName','itemCode'])

for key in list(identity_df.columns):
    identity_df[key] = [tomatoIdentityList[i][key] for i in range(len(tomatoIdentityList))]

identity_df

,userId,facilityId,addressName,itemCode
0,PFS_0000001,PFS_0000001_01,경상남도 사천시,080300
1,PFS_0000002,PFS_0000002_01,전라북도 김제시,080300
2,PFS_0000003,PFS_0000003_01,경상남도 함안군,080300
3,PFS_0000004,PFS_0000004_01,경상남도 진주시,080300
4,PFS_0000006,PFS_0000006_01,전라북도 순창군,080300
...,...,...,...,...
100,sk0081,sk0081_01,세종특별자치시 연동면,080300
101,sk0085,sk0085_01,세종특별자치시 연동면,080300
102,sk0091,sk0091_01,세종특별자치시 연동면,080300
103,sk0104,sk0104_01,세종특별자치시 연동면,080300


In [20]:
identity_df.to_csv('data/identity.csv')

## 토마토 농가 작기 정보 수집
작기란? 한 작물의 생육기간을 1작기로 함. 예) 벼 150∼180일.

![](img/CroppingSeasonDataList.png)

In [5]:
key_list = ['userId',
 'statusCode',
 'statusMessage',
 'croppingSerlNo',
 'croppingSeasonName',
 'croppingDate',
 'croppingEndDate',
 'croppingSystem',
 'itemCode',
 'cultivationArea',
 'calCultivationArea',
 'plantNum',
 'calPlantNum',
 'stemSlabNum',
 'planSlabNum',
 'plantDensity',
 'standardPlantDensity',
 'floodlightDec',
 'leafArea',
 'stndTemp',
 'stndWeight',
 'stndSolar',
 'stndMeta']

In [6]:
getCroppingSeasonDataList = 'http://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getCroppingSeasonDataList/'+serviceKey+'/'
CroppingSeasonDataList = []
cropping_df = pd.DataFrame(columns=key_list)

for i in range(len(identity_df)):
    response = requests.get(getCroppingSeasonDataList+identity_df.loc[i,'userId'])
    temp = response.text
    data = json.loads(temp)
    data[0]['userId'] = identity_df.loc[i,'userId']
    CroppingSeasonDataList.append(data[0])

for key in key_list:
    cropping_df[key] = [CroppingSeasonDataList[i][key] for i in range(len(identity_df))]
    
cropping_df

,userId,statusCode,statusMessage,croppingSerlNo,croppingSeasonName,croppingDate,croppingEndDate,croppingSystem,itemCode,cultivationArea,...,stemSlabNum,planSlabNum,plantDensity,standardPlantDensity,floodlightDec,leafArea,stndTemp,stndWeight,stndSolar,stndMeta
0,PFS_0000001,00,NORMAL_CODE,79,데프니스_20150902,2015-09-02 00:00:00,2016-12-31 00:00:00,B0004000000000000002,080300,1180.0,...,4.0,4.0,2.44,2.5,0.80,0.0,20.0,180.0,150.0,130.0
1,PFS_0000002,00,NORMAL_CODE,80,다볼_20150714,2015-07-14 00:00:00,2016-12-31 00:00:00,B0004000000000000002,080300,3400.0,...,4.0,4.0,2.37,2.5,0.70,0.0,20.0,180.0,150.0,150.0
2,PFS_0000003,00,NORMAL_CODE,81,데프니스_20150902,2015-09-02 00:00:00,2016-12-31 00:00:00,B0004000000000000002,080300,1500.0,...,2.0,4.0,2.40,2.5,0.70,0.0,18.0,180.0,100.0,130.0
3,PFS_0000004,00,NORMAL_CODE,82,데프니스_20150720,2015-07-20 00:00:00,2016-06-30 00:00:00,B0004000000000000002,080300,1200.0,...,5.0,5.0,2.36,2.5,0.65,0.0,18.0,180.0,100.0,130.0
4,PFS_0000006,00,NORMAL_CODE,69,다볼_20150625,2015-06-25 00:00:00,2016-12-31 00:00:00,B0004000000000000002,080300,3000.0,...,2.0,2.0,2.23,6.8,0.70,0.0,20.0,180.0,150.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,sk0081,00,NORMAL_CODE,41,알렉산더_20160202,2016-02-02 00:00:00,2016-12-31 00:00:00,B0004000000000000001,080300,3000.0,...,5.0,5.0,2.73,2.5,0.70,0.0,18.0,180.0,100.0,130.0
101,sk0085,00,NORMAL_CODE,10,마미리오_20160120,2016-01-20 00:00:00,2016-12-31 00:00:00,B0004000000000000001,080300,3000.0,...,5.0,5.0,2.73,2.5,0.70,0.0,18.0,180.0,100.0,130.0
102,sk0091,00,NORMAL_CODE,45,244_20160203,2016-02-03 00:00:00,2016-12-31 00:00:00,B0004000000000000001,080300,3000.0,...,5.0,5.0,2.73,2.5,0.70,0.0,18.0,180.0,100.0,130.0
103,sk0104,00,NORMAL_CODE,326,티티찰_2016,2016-07-20 00:00:00,2016-12-31 00:00:00,B0004000000000000002,080300,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN


In [32]:
cropping_df[cropping_df['croppingSystem'].isna()]

,index,userId,statusCode,statusMessage,croppingSerlNo,croppingSeasonName,croppingDate,croppingEndDate,croppingSystem,itemCode,...,stemSlabNum,planSlabNum,plantDensity,standardPlantDensity,floodlightDec,leafArea,stndTemp,stndWeight,stndSolar,stndMeta
22,23,PF_0000018,0,NORMAL_CODE,2007,2019_토마토,2019-07-29 00:00:00,2020-06-30 00:00:00,NaN,80300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,48,PF_0002525,0,NORMAL_CODE,2116,2019년_토마토(대장금)_01,2019-08-08 00:00:00,2019-12-13 00:00:00,NaN,80300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,58,PF_0006023,0,NORMAL_CODE,2308,2019년_토마토,2019-12-21 00:00:00,2020-06-15 00:00:00,NaN,80300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,59,PF_0006024,0,NORMAL_CODE,2319,2019년_방울토마토,2019-10-05 00:00:00,2020-06-15 00:00:00,NaN,80600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,60,PF_0006025,0,NORMAL_CODE,2309,2019년_토마토,2019-11-05 00:00:00,2020-06-15 00:00:00,NaN,80300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,61,PF_0006042,0,NORMAL_CODE,2303,2019년_토마토(토사마),2019-11-05 00:00:00,2020-05-30 00:00:00,NaN,80300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,62,PF_0006043,0,NORMAL_CODE,2322,2019년_토마토,2019-10-02 00:00:00,2020-06-01 00:00:00,NaN,80300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,63,PF_0006044,0,NORMAL_CODE,2304,2019년_토마토(데프니스),2019-09-06 00:00:00,2020-07-01 00:00:00,NaN,80300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,64,PF_0006045,0,NORMAL_CODE,2326,2019년_방울토마토,2019-09-15 00:00:00,2020-06-30 00:00:00,NaN,80600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,65,PF_0006046,0,NORMAL_CODE,2760,2019_토마토,2019-08-04 00:00:00,2020-06-30 00:00:00,NaN,80300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
cropping_df[cropping_df['croppingSerlNo']==2304]

,index,userId,statusCode,statusMessage,croppingSerlNo,croppingSeasonName,croppingDate,croppingEndDate,croppingSystem,itemCode,...,stemSlabNum,planSlabNum,plantDensity,standardPlantDensity,floodlightDec,leafArea,stndTemp,stndWeight,stndSolar,stndMeta
62,63,PF_0006044,0,NORMAL_CODE,2304,2019년_토마토(데프니스),2019-09-06 00:00:00,2020-07-01 00:00:00,NaN,80300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
cropping_df=cropping_df.drop(cropping_df[cropping_df['itemCode']=='080400'].index)

In [8]:
cropping_df[cropping_df['itemCode']=='080400']

,userId,statusCode,statusMessage,croppingSerlNo,croppingSeasonName,croppingDate,croppingEndDate,croppingSystem,itemCode,cultivationArea,...,stemSlabNum,planSlabNum,plantDensity,standardPlantDensity,floodlightDec,leafArea,stndTemp,stndWeight,stndSolar,stndMeta


In [9]:
cropping_df.reset_index(inplace=True)

In [18]:
cropping_df = pd.read_csv('data/cropping_user.csv', index_col=0)
cropping_df.head()

,index,userId,statusCode,statusMessage,croppingSerlNo,croppingSeasonName,croppingDate,croppingEndDate,croppingSystem,itemCode,...,stemSlabNum,planSlabNum,plantDensity,standardPlantDensity,floodlightDec,leafArea,stndTemp,stndWeight,stndSolar,stndMeta
0,0,PFS_0000001,0,NORMAL_CODE,79,데프니스_20150902,2015-09-02 00:00:00,2016-12-31 00:00:00,B0004000000000000002,80300,...,4.0,4.0,2.44,2.5,0.80,0.0,20.0,180.0,150.0,130.0
1,1,PFS_0000002,0,NORMAL_CODE,80,다볼_20150714,2015-07-14 00:00:00,2016-12-31 00:00:00,B0004000000000000002,80300,...,4.0,4.0,2.37,2.5,0.70,0.0,20.0,180.0,150.0,150.0
2,2,PFS_0000003,0,NORMAL_CODE,81,데프니스_20150902,2015-09-02 00:00:00,2016-12-31 00:00:00,B0004000000000000002,80300,...,2.0,4.0,2.40,2.5,0.70,0.0,18.0,180.0,100.0,130.0
3,3,PFS_0000004,0,NORMAL_CODE,82,데프니스_20150720,2015-07-20 00:00:00,2016-06-30 00:00:00,B0004000000000000002,80300,...,5.0,5.0,2.36,2.5,0.65,0.0,18.0,180.0,100.0,130.0
4,4,PFS_0000006,0,NORMAL_CODE,69,다볼_20150625,2015-06-25 00:00:00,2016-12-31 00:00:00,B0004000000000000002,80300,...,2.0,2.0,2.23,6.8,0.70,0.0,20.0,180.0,150.0,150.0


In [51]:
cropping_df.to_csv('data/cropping_user.csv')

## ★작기별 제어정보

In [10]:
key_list=['statusCode',
 'statusMessage',
 'fatrCode',
 'facilityId',
 'measDate',
 'fldCode',
 'sectCode',
 'itemCode',
 'senVal',
 'ymd',
 'cntCollect',
 'num',
 'totalRows',
 'totalPage',
 'currentPage']

In [11]:
for i in tqdm(range(58,105)):
    cropping_data = pd.DataFrame(columns=key_list)

    croppingSerlNo=cropping_df['croppingSerlNo'][i]
    getCroppingSeasonManlDataListResponse = 'http://www.smartfarmkorea.net/Agree_WS/webservices/CropseasonRestService/getCroppingSeasonManlDataList/'+serviceKey+'/'+str(croppingSerlNo)+'/'
    response = requests.get(getCroppingSeasonManlDataListResponse+str(1))
    temp = response.text
    data = json.loads(temp)

    totalPage=data[0]['totalPage']
    if totalPage:
        print(data[0]['facilityId'])
        for j in tqdm(range(int(totalPage)-1)):
            response = requests.get(getCroppingSeasonManlDataListResponse+str(j+1))
            temp = response.text
            temp_data = json.loads(temp)

            data.extend(temp_data)

        for key in key_list:
            cropping_data[key] = [data[i][key] for i in range(len(data))]

        cropping_data.to_csv('data/cropping/{}.csv'.format(data[0]['facilityId'][:-3]))

PF_0006024_01



PF_0006025_01



PF_0006042_01



PF_0006043_01



PF_0006044_01


...


PF_0010076_01



PF_0010077_01



PF_0020353_01



PF_0020356_01


KeyError: 104

In [16]:
file_list=os.listdir(r'C:\Users\youjeongsue\Desktop\study\4-2\스마트팜(소융캡&창종설)\data\cropping')
len(file_list)

59

## ★작기별 생육정보

In [14]:
key_list = ['examinDe',
 'fcltyId',
 'itemCode',
 'croppingSerlNo',
 'examinMasterSn',
 'examinSn',
 'sampleNo',
 'examinIemCode',
 'examinIemNm',
 'examinIemValue',
 'examinIemUnit']

In [33]:
#print된 것들은 cultivate data가 없는 것들..! 최종에서 빼야겠다.
for i in tqdm(range(len(cropping_df))):
    try:
        cultivate_data = pd.DataFrame(columns=key_list)

        croppingSerlNo=cropping_df['croppingSerlNo'][i]
        getCultivateDataListResponse = 'http://www.smartfarmkorea.net/Agree_WS/webservices/CropseasonRestService/getCultivateDataList/'+serviceKey+'/'+str(croppingSerlNo)+'/'
        response = requests.get(getCultivateDataListResponse)
        temp = response.text
        data = json.loads(temp)

        for key in key_list:
            cultivate_data[key] = [data[i][key] for i in range(len(data))]

        cultivate_data.to_csv('data/cultivate/{}.csv'.format(data[0]['fcltyId'][:-3]))
    except:
        print(croppingSerlNo)

2304
2326
2760
2601
2640
2575
2576
3269
2110
2568
3303
3300



## ★작기별 환경정보

In [54]:
key_list = ['fatrCode',
 'facilityId',
 'measDate',
 'fldCode',
 'sectCode',
 'itemCode',
 'senVal',
 'ymd',
 'cntCollect',
 'num',
 'totalRows',
 'totalPage',
 'currentPage']

In [60]:
#seasonenv_data가 없는 것들. 최종에서 빼야 함
no_seasonenv_data = []
#41,82해야함
# for i in tqdm(range(83, 105)):
for i in tqdm(range(len(cropping_df))):
    try:
        seasonenv_data = pd.DataFrame(columns=key_list)

        croppingSerlNo=cropping_df['croppingSerlNo'][i]
        getCroppingSeasonEnvDataLis = 'http://www.smartfarmkorea.net/Agree_WS/webservices/CropseasonRestService/getCroppingSeasonEnvDataList/'+serviceKey+'/'+str(croppingSerlNo)+'/'
        response = requests.get(getCroppingSeasonEnvDataLis+str(1))
        temp = response.text
        data = json.loads(temp)

        totalPage=data[0]['totalPage']
        if totalPage:
            print(data[0]['facilityId'])
            for j in tqdm(range(int(totalPage)-1)):
                response = requests.get(getCroppingSeasonEnvDataLis+str(j+1))
                temp = response.text
                temp_data = json.loads(temp)

                data.extend(temp_data)

            for key in key_list:
                seasonenv_data[key] = [data[i][key] for i in range(len(data))]

            seasonenv_data.to_csv('data/seasonenv/{}.csv'.format(data[0]['facilityId'][:-3]))
    except:
        no_seasonenv_data.append(croppingSerlNo)

PF_0010075_01



PF_0010076_01



PF_0010077_01



PF_0020353_01



PF_0020356_01


## 토마토 생육 정보

![](img/CultivateDataList.png)

![](img/CultivateData.png)

In [82]:
key_list=['statusCode',
 'statusMessage',
 'measDate',
 'userId',
 'itemCode',
 'sampleNum',
 'growLength',
 'flowerTop',
 'stemDiameter',
 'leavesLength',
 'leavesWidth',
 'leavesNum',
 'flowerPosition',
 'fruitsPosition',
 'fruitsNum',
 'harvestPosition',
 'ped',
 'solarCorrection',
 'fruitsWeight']

In [92]:
all_cultivate_df = pd.DataFrame()
for i in range(len(season_df)):
    cultivate_df = pd.DataFrame(columns=key_list)
    
    getCultivateDataList = 'http://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getCultivateDataList/'+serviceKey+'/'+cropping_df.loc[i,'userId']+'/'+str(cropping_df.loc[i,'croppingSerlNo'])+'/'+cropping_df.loc[i,'croppingDate'].split(' ')[0]+'/'+cropping_df.loc[i,'croppingEndDate'].split(' ')[0]
    response = requests.get(getCultivateDataList)
    temp = response.text
    CultivateDataList = json.loads(temp)
    
    for key in key_list:
        cultivate_df[key] = [CultivateDataList[j][key] for j in range(len(CultivateDataList))]
        
    cultivate_df.to_csv('data/users/{}.csv'.format(cropping_df.loc[i,'userId']))
    
    temp = cultivate_df.copy()
    temp['userId']=[cropping_df.loc[i,'userId']]*len(cultivate_df)
    all_cultivate_df = all_cultivate_df.append(temp)
    
all_cultivate_df.to_csv('data/all_cultivate_df.csv')

In [95]:
all_cultivate_df

,statusCode,statusMessage,measDate,userId,itemCode,sampleNum,growLength,flowerTop,stemDiameter,leavesLength,leavesWidth,leavesNum,flowerPosition,fruitsPosition,fruitsNum,harvestPosition,ped,solarCorrection,fruitsWeight
0,00,NORMAL_CODE,2016-03-13 00:00:00,PFS_0000001,080300,2,20.0,19.0,12.00,46.0,42.0,22.0,3.0,3.0,3.0,0,0,0,0
1,None,None,2016-03-13 00:00:00,PFS_0000001,080300,3,20.0,19.0,12.00,46.0,42.0,22.0,3.0,3.0,3.0,0,0,0,0
2,None,None,2016-03-06 00:00:00,PFS_0000001,080300,3,20.0,20.0,12.00,46.0,41.0,20.0,3.0,3.0,3.0,0,0,0,0
3,None,None,2016-03-06 00:00:00,PFS_0000001,080300,4,20.0,20.0,12.00,46.0,41.0,20.0,3.0,3.0,3.0,0,0,0,0
4,None,None,2016-02-14 00:00:00,PFS_0000001,080300,1,18.0,20.0,12.00,47.0,41.0,18.0,3.0,3.0,3.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,None,None,2016-05-01 00:00:00,sk0118,080300,1,21.0,12.0,7.15,31.0,30.0,17.0,6.4,5.4,21.0,0,0,0,0
84,None,None,2016-05-15 00:00:00,sk0118,080300,2,14.0,5.0,8.92,32.0,29.0,15.0,8.8,7.6,20.0,0,0,0,0
85,None,None,2016-06-19 00:00:00,sk0118,080300,3,0.0,7.0,6.65,30.0,26.0,12.0,11.0,11.0,15.0,0,0,0,0
86,None,None,2016-07-03 00:00:00,sk0118,080300,2,0.0,10.0,7.96,29.0,27.0,16.0,9.0,9.0,9.0,0,0,0,0
